# Before running the pipeline, change:
## `Runtime` -->  `Change runtime type` --> `GPU`

In [ ]:
#@title Links that are subject to updates

WEIGHTS="https://d.aibird.me/8fdadb26.pt"
CLASSES="https://d.aibird.me/83b3976c.txt"

In [ ]:
%%shell
sudo apt -qq update
sudo apt -qq install ffmpeg libsm6 libxext6  -y

In [ ]:
%%shell
wget -q https://downloads.rclone.org/v1.58.0/rclone-v1.58.0-linux-amd64.zip
unzip rclone-v1.58.0-linux-amd64.zip
mv rclone-v1.58.0-linux-amd64/rclone .

In [ ]:
from getpass import getpass
RCLONE_CONFIG = getpass('Download link from bitwarden: ')

In [ ]:
! wget -q "$RCLONE_CONFIG" -O rclone.conf.gpg
! echo $(python -c "from getpass import getpass; print(getpass())") | \
  gpg -d --batch --yes --passphrase-fd 0 rclone.conf.gpg > rclone.conf

In [ ]:
%%shell
mkdir "downloaded_$(date +%m-%d-%Y)"

./rclone copy gdrive: "downloaded_$(date +%m-%d-%Y)" --config "rclone.conf" \
 --transfers 32 -P >> out.log

./rclone size "downloaded_$(date +%m-%d-%Y)"

In [ ]:
%%shell
pip -q install label_studio_converter lxml Pillow "PyYAML>=5.3.1"

In [ ]:
%%shell
git clone https://github.com/ultralytics/yolov5.git
pip -q install -r yolov5/requirements.txt

wget "$WEIGHTS" -qO "weights.pt"

In [ ]:
%%shell
python yolov5/detect.py --source "downloaded_$(date +%m-%d-%Y)" \
--weights "weights.pt" --save-txt --save-conf

## While the process is running, run the following commands on the cluster:

```sh
cd /share/$GROUP/$USER/megadetector_picam

set NEW_FOLDER_NAME="downloaded_`date +%m-%d-%Y`"

./rclone move -P --transfers 32 --stats-one-line --max-depth 1 --filter "+ *.jpg" --filter "- *" gdrive:"picam" gdrive:"picam-downloaded/$NEW_FOLDER_NAME"
```

In [ ]:
%%shell
mkdir $folder/images
mv $folder/*.jpg $folder/images

wget "$CLASSES" -qO $folder/classes.txt
wget "https://d.aibird.me/ab537c0f.py" -qO "yolo.py"

In [ ]:
%%shell
python yolo.py -i "downloaded_$(date +%m-%d-%Y)" -o output.json

In [ ]:
from datetime import date
from glob import glob

folder = sorted(glob('yolov5/runs/detect/*'))[-1]
ts = f'downloaded_{date.today().strftime("%m-%d-%Y")}'

In [ ]:
from yolo import convert_yolo_to_ls

convert_yolo_to_ls(input_dir=folder, out_file='output.json',
    to_name='image', from_name='label', out_type="annotations",
    image_root_url=f'https://ls.aibird.me/data/local-files/?d=picam/{ts}/with_detections',
    image_ext='.jpg')

## <---- Download `output.json`

## On the local server, run the following commands:

```sh
cd "/home/ubuntu/apps/label-studio"

source .env && docker run --rm -v "$PWD/pg_backups:/backups" -e POSTGRES_HOST=${_POSTGRES_HOST} -e POSTGRES_DB=${POSTGRE_NAME} -e POSTGRES_USER=${POSTGRE_USER} -e POSTGRES_PASSWORD=${POSTGRE_PASSWORD} prodrigestivill/postgres-backup-local /backup.sh

rclone copy pg_backups pg_backups: -P --stats-one-line -L
```

```sh
NEW_FOLDER_NAME="local-files/picam/downloaded_$(date +%m-%d-%Y)"

rclone copy gdrive:"downloaded_$(date +%m-%d-%Y)" "$NEW_FOLDER_NAME" -P --stats-one-line --transfers 32
```

```sh
python sync_picam.py
```

## Import the downloaded `output.json` into the label-studio project


1. Open the project, then click `Import`.

![import](https://d.aibird.me/af44a951.png)


2. Drag and drop the `output.json` file, wait few seconds for it to be uploaded, then click `import`.

![import2](https://d.aibird.me/8d399e66.png)
